## Import libraries

In [2]:
import sys
import threading
from pathlib import Path
sys.path.append(str(Path().resolve().parent))
from simworld.config import Config
from simworld.communicator.communicator import Communicator
from simworld.communicator.unrealcv import UnrealCV
from simworld.traffic.controller.traffic_controller import TrafficController
from simworld.traffic.base.traffic_signal import TrafficSignalState
from simworld.llm.a2a_llm import A2ALLM
from simworld.llm.base_llm import BaseLLM
from simworld.map.map import Map
from simworld.agent.humanoid import Humanoid
from simworld.utils.vector import Vector
from simworld.local_planner.local_planner import LocalPlanner

FileNotFoundError: [Errno 2] No such file or directory

## Init

In [ ]:
communicator = Communicator(UnrealCV())

In [ ]:
import os
os.environ['OPENAI_API_KEY'] = '<your_openai_api_key_here>'  # Replace with your actual OpenAI API key

## Agent

In [ ]:
class Agent():
    def __init__(self, goal):
        self.goal = goal
        self.llm = BaseLLM("gpt-4o")
        self.system_prompt = f"You are an intelligent agent in a 3D world. Your goal is to: {self.goal}."

    def action(self, obs):
        prompt = f"{self.system_prompt}\n You are currently at: {obs}\nWhat is your next goal?"
        action = self.llm.generate_text(system_prompt=self.system_prompt, user_prompt=prompt)
        return action

In [ ]:
class Environment:
    def __init__(self, communicator, config=Config()):
        self.communicator = communicator
        self.agent = None
        self.target = None
        self.action_planner = None
        self.config = config
        self.action_planner_llm = A2ALLM(model_name="gpt-4o-mini")
        self.map = Map(config)
        self.map.initialize_map_from_file(roads_file= os.path.join('../data/sample_data/road.json')) # use default map

    def reset(self):
        """Clear the UE scene and (re)spawn the humanoid and target."""
        # Clear spawned objects
        self.communicator.clear_env()

        # Blueprint path for the humanoid agent to spawn in the UE level
        agent_bp = "/Game/TrafficSystem/Pedestrian/Base_User_Agent.Base_User_Agent_C"

        # Initial spawn position and facing direction for the humanoid (2D)
        spawn_location = Vector(0, 0)
        spawn_forward = Vector(0, 1)
        self.agent = Humanoid(
            communicator=self.communicator,
            position=spawn_location,
            direction=spawn_forward,
            config=self.config,
            map=self.map
        )

        self.action_planner = LocalPlanner(
            agent=self.agent,
            model=self.action_planner_llm,
            rule_based=False)

        # Spawn the humanoid agent in the Unreal world
        # NOTE: name is ignored for humanoid type, but required by the API.
        self.communicator.spawn_agent(self.agent, name=None, model_path=agent_bp, type="humanoid")

        # Cache the generated UE actor name
        self.agent_name = self.communicator.get_humanoid_name(self.agent.id)

        # Define a target position the agent is encouraged to move toward (example value)
        self.target = Vector(1000, 0)

        # Return initial observation
        # observation = self.communicator.get_camera_observation(self.agent.camera_id, "lit")
        observation = self.communicator.unrealcv.get_location(self.agent_name)
        return observation

    def step(self, action):
        """Use action planner to execute the given action."""
        # Parse the action text and map it to the action space
        primitive_actions = self.action_planner.parse(action)
        self.action_planner.execute(primitive_actions)

        # Get current location from UE (x, y, z) and convert to 2D Vector
        loc_3d = self.communicator.unrealcv.get_location(self.agent_name)
        # loc_3d is a numpy array; explicitly use x, y to build our 2D Vector
        location = Vector(loc_3d[0], loc_3d[1])

        # Camera observation for RL
        observation = location

        # Reward: negative Euclidean distance in 2D plane
        reward = -location.distance(self.target)

        return observation, reward

## Gym-like Interface

In [ ]:
agent = Agent(goal='Go to (1700, -1700) and pick up GEN_BP_Box_1_C.')
env = Environment(communicator)

obs = env.reset()

# Roll out a short trajectory
for _ in range(100):
    action = agent.action(obs)
    obs, reward = env.step(action)
    print(f"obs: {obs}, reward: {reward}")
    # Plug this into your RL loop / logging as needed

In [ ]:
communicator.disconnect()